In [5]:
import pygame 
from modsim import *
import numpy as np

%matplotlib notebook



In [6]:
"""This is our starting condition. This holds all of our constants and initial conditions."""
condition = Condition(angle = 50, #in degrees
                     mass = .0027, #in kilograms
                     diameter = .04, #in meters 
                     rho = 1.16, #air density in kg/m^3
                     C_d = .445, #unitless
                     cannon_length = .91,
                     velocity = 90,
                     duration = 20,
                     g=9.81,
                     omega = 0.00012) #meters

"""This function will make our system with all of a state that contains all of the initial conditions."""
def make_system(condition):
    unpack(condition)
    theta = np.deg2rad(angle)
    vx, vy = pol2cart(theta, velocity)
    x, y = pol2cart(theta, cannon_length)
    init = State(x=x, y=y, vx=vx, vy=vy)
    area = np.pi * (diameter/2)**2
    ts = linspace(0, duration, 101)
    return System(init=init, g=g, mass=mass,
        area=area, rho=rho, C_d=C_d, ts=ts, omega=omega)

"""This function will calculate the derivatives of everything changning in our state."""
def slope_func(state,t,system):
    
    x,y,vx,vy = state
    unpack(system)
    
    v=Vector(vx,vy)
    
    a_grav = Vector(0,-g) #turning gravity into a vector
    a_drag = (-rho*v.mag*v*C_d*area/2)/mass #calculating acceleration due to drag
   
    
    a_magnus_mag = (abs((omega)*(v.mag**2)))/mass # calculating magnitude of magnus acceleration
    a_magnus_angle = v.angle + np.pi/2 # making angle of magnus acceleration perpendecular to direction of velocity
    magnus_x, magnus_y = pol2cart(a_magnus_angle * UNITS.radian,a_magnus_mag) # turning magnitude and angle into x and y components
    a_magnus = Vector(magnus_x, magnus_y) #turning components into a vector
    
    a = a_grav + a_drag + a_magnus #adding all accerlations together as vectors
    
    
    
    return vx, vy, a.x, a.y #dot operators are used for accleration because it is a vector


"""This function will use the results generated by run_odeint and return the range of the launch"""
def interpolate_range(results):
    
    xs = results.x
    ys = results.y
    
    t_end = ys.index[-1]
    if ys[t_end] > 0:
        print("The simulation did not run long enough for the ball to hit the ground. Make the duration longer and try again.")
        return -1
        #msg = """The simulation did not run long enough for the ball to hit the ground. Make the duration longer and try again."""
        #raise ValueError(msg)
        
    Y = interpolate(ys, kind = 'cubic')
    t_peak = ys.argmax() #this is the time that the ball is the highest in the air
    descent = ys.loc[t_peak:]
    T = interp_inverse(descent, kind ='cubic')
    
    t_land = T(0)
    X = interpolate(xs, kind='cubic')
    
    return X(t_land)

def plot_trajectory(x,y):
    newfig()
    plot(x,y)
    decorate(title="Trajectory Plot",
            ylabel = "Height",
            xlabel = "Distance")
    
def run_simulation(condition, slope_func): 
    system=make_system(condition)
    run_odeint(system,slope_func)
    
    return system

def Optimize_duration(x,y):
    xs = x
    ys = y
    
    t_end = ys.index[-1]
    if ys[t_end] > 0:
        msg = """The simulation did not run long enough for the ball to hit the ground. Make the duration longer and try again."""
        raise ValueError(msg)
        
    Y = interpolate(ys, kind = 'cubic')
    t_peak = ys.argmax() #this is the time that the ball is the highest in the air
    descent = ys.loc[t_peak:]
    T = interp_inverse(descent, kind ='cubic')
    
    t_end = T(0)
    
    return t_end
    
def run_corrected_simulation(condition, slope_func):
    system = run_simulation(condition, slope_func)
    t_end = Optimize_duration(system.results.x, system.results.y)
    condition.set(duration = t_end)
    system = run_simulation(condition, slope_func)
    #plot_trajectory(system.results.x, system.results.y)
    
    return system
    



In [3]:
def resize_image(image, dimensions):
    new_image = pygame.transform.scale(image, dimensions)
    #(Surface, (width, height), DestSurface = None)
    return new_image



# Define some colors
import sys
from pygame.locals import *
pygame.init()

# Set the width and height of the screen [width, height] 
# (0,0) is upper-right corner
screen_size = (1100, 700)
screen = pygame.display.set_mode(screen_size)
pygame.display.set_caption("Le Window")

# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()


#load pictures
Ricky = pygame.image.load("RickySprite.png")
Florida = pygame.image.load("Florida.jpg")
newFlorida = resize_image(Florida,screen_size)
Cannon = pygame.image.load("Cannon.png")
Ball_orig = pygame.image.load("Ball.png")
Ball = resize_image(Ball_orig, (20,20))

#Other Stuff
pressed = pygame.key.get_pressed()
font = pygame.font.SysFont('Arial', 80, True, False)
screen_rect = screen.get_rect()
Cannon_rect = Cannon.get_rect(center=([20,555]))
angle = 0
t = 0
shoot = False
initialize = False
magnification_x = 50
magnification_y = 50

# -------- Main Program Loop -----------
while not done:
    for event in pygame.event.get():
        #Quit Game code
        if event.type == pygame.QUIT:
            done = True
         
        #Check if key is pressed
        elif (event.type == KEYDOWN):
            #Get key for shoot    
            if (event.key == K_SPACE):
                shoot = True
                t = 0
                cannon_tip_x, cannon_tip_y = pol2cart(angle*pi/180, 180) 
                condition.set(angle=angle)
                system = run_corrected_simulation(condition, slope_func)
                xs = system.results.x
                ys = system.results.y

                
            #Get key for angle
       
            if (event.key == K_UP):
                angle += 5
            if (event.key == K_DOWN):
                angle-= 5
            

        
    #Animate background and Ricky
    screen.blit(newFlorida,(0,0))
    screen.blit(Ricky,(10,410))
        
    #Find ball position
    if shoot == True:
        pygame.draw.rect(screen,(0,255,0), [100,100,100,100])
        start_x = 20 + cannon_tip_x
        start_y = 555 - cannon_tip_y
        
        #magnification_x = start_x/xs[xs.index[0]]
        #magnification_y = start_y/ys[ys.index[0]]
        
        offset_x =  start_x - xs[xs.index[0]]*magnification_x 
        offset_y = start_y + ys[ys.index[0]]*magnification_y
        
        x = offset_x + xs[xs.index[t]]*magnification_x 
        y = offset_y - ys[ys.index[t]]*magnification_y
        
        screen.blit(Ball, (x, y))
        
        if t < len(system.results) - 1:
            t += 1
        else:
            shoot = False
            
        

        
        #Animate Cannon
    newCannon = pygame.transform.rotate(Cannon,angle)
    Cannon_rect = newCannon.get_rect(center=Cannon_rect.center) #LOOK HERE ERIKA
    screen.blit(newCannon, Cannon_rect)
        
        #Make angle sign
    pygame.draw.rect(screen, (0,0,100), [940, 10, 140, 100])
    pygame.draw.rect(screen, (255,255,255), [950, 20, 120, 80])
    angle_string = str(int(angle)
    angle_degrees = angle_string+ u'\u00b0'
    angle_display = font.render(angle_degrees, True, (0,0,0))
    screen.blit(angle_display, (960,20))
    pygame.display.flip()
    clock.tick(20) # Limit to # frames per second
            
    
# Close the window and quit.
pygame.quit()
#sys.exit()


In [4]:
 #sweep magnus force for optimal angles

In [ ]:
#show trajectory for 0 and 25 degrees
#have PyGame output range